# Vendor Analysis: Lending Scorecard

In [2]:
import pandas as pd

df = pd.read_csv("../data/telegram_data.csv")
df.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,Sheger online-store,@Shageronlinestore,7383,💥Miralux Hot plate\n ባለሁለት ምድጃ ስቶቭ\n\n 💯o...,2025-06-19 06:31:31+00:00,data/photos/@Shageronlinestore_7383.jpg
1,Sheger online-store,@Shageronlinestore,7382,💥7pcs glass water set\n\n✔️ አንድ ማራኪ ጆግና 6 መጠጫ ...,2025-06-18 11:19:11+00:00,data/photos/@Shageronlinestore_7382.jpg
2,Sheger online-store,@Shageronlinestore,7381,NaN,2025-06-18 11:19:11+00:00,data/photos/@Shageronlinestore_7381.jpg
3,Sheger online-store,@Shageronlinestore,7380,NaN,2025-06-18 11:19:11+00:00,data/photos/@Shageronlinestore_7380.jpg
4,Sheger online-store,@Shageronlinestore,7379,NaN,2025-06-18 11:19:11+00:00,data/photos/@Shageronlinestore_7379.jpg


In [ ]:
from transformers import pipeline
ner_pipeline = pipeline("token-classification", model="Davlan/bert-base-multilingual-cased-ner-hrl", aggregation_strategy="simple")

In [5]:
import numpy as np
from datetime import timedelta

def analyze_vendors(df, ner_pipeline):
    # df['timestamp'] = pd.to_datetime(df['timestamp'])
    vendors = df['vendor'].unique()

    vendor_metrics = []

    for vendor in vendors:
        vendor_df = df[df['vendor'] == vendor].copy()
        vendor_df.sort_values("timestamp", inplace=True)

        # Posts/week
        days_range = (vendor_df['timestamp'].max() - vendor_df['timestamp'].min()).days or 1
        posts_per_week = len(vendor_df) / (days_range / 7)

        # Views
        avg_views = vendor_df['views'].mean()
        top_post = vendor_df.loc[vendor_df['views'].idxmax()]
        top_product = top_post['text']
        top_views = top_post['views']

        # Price extraction
        prices = []
        for text in vendor_df['text']:
            preds = ner_pipeline(text)
            price_tokens = [p['word'] for p in preds if 'PRICE' in p['entity']]
            joined = " ".join(price_tokens).replace(",", "").replace("ETB", "")
            for token in joined.split():
                try:
                    val = float(token)
                    prices.append(val)
                except:
                    continue

        avg_price = np.mean(prices) if prices else 0.0

        # Lending Score (simple weights)
        score = (avg_views * 0.5) + (posts_per_week * 0.5)

        vendor_metrics.append({
            "vendor": vendor,
            "Posts/Week": round(posts_per_week, 2),
            "Avg Views/Post": round(avg_views, 2),
            "Avg Price (ETB)": round(avg_price, 2),
            "Top Product": top_product,
            "Top Views": top_views,
            "Lending Score": round(score, 2)
        })

    return pd.DataFrame(vendor_metrics)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys 
sys.path.append(os.path.abspath(os.path.join('..')))
from transformers import pipeline
from scripts.vendor_scorecard_engine import score_vendors

t:\10-academy\Week4\Telegram-e-commerce-analytics\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:

# Load scraped Telegram data
df = pd.read_csv("../data/telegram_data.csv")

In [23]:

# Normalize column names
df.columns = df.columns.str.lower().str.strip()
print("Columns after load:", df.columns.tolist())


Columns after load: ['channel title', 'channel username', 'id', 'message', 'date', 'media path']


In [24]:
# Rename columns to match expectations in score_vendors
df.rename(columns={
    'channel title': 'vendor',
    'message': 'text',
    'date': 'timestamp'
}, inplace=True)

# Simulate 'views' since missing from data
np.random.seed(42)
df['views'] = np.random.randint(100, 5000, size=len(df))

In [25]:

# Parse timestamps and drop rows with invalid timestamps
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
if df['timestamp'].isnull().any():
    print(f"Dropping {df['timestamp'].isnull().sum()} rows with invalid timestamps")
    df = df.dropna(subset=['timestamp'])

# Remove timezone info if present (to avoid warnings in score_vendors)
df['timestamp'] = df['timestamp'].dt.tz_localize(None)

In [27]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load your model (replace with your model path or Hugging Face model name)
model_name = "Davlan/afro-xlmr-base"  # or "models/saved_model_dir" for local fine-tuned model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Now define the NER pipeline
ner_pipeline = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [ ]:
score_df = score_vendors(df, ner_pipeline)

# Save results
score_df.to_csv("../data/vendor_scorecard.csv", index=False)

# Display head of results
print(score_df.head())
